# Quality control routines for PNBOIA

In [1]:
import pandas as pd
%load_ext autoreload
%autoreload 2
import pnboia_qc.pnboia_limits as pnboia_limits
from pnboia_qc.qc_checks import QCChecks


# conexao banco
from dotenv import load_dotenv
import pandas.io.sql as sqlio
import psycopg2
import os

load_dotenv()


True

## Load PNBOIA data

In [2]:
df = pd.read_excel('../raw_data/po.xlsx')

FileNotFoundError: [Errno 2] No such file or directory: '../raw_data/po.xlsx'

In [3]:
df.head()

,#ano,mes,dia,hora,lon,lat,battery,wspd1,gust1,wdir1,...,cdir1,cvel2,cdir2,cvel3,cdir3,wvht,wmax,dpd,mwd,spred
0,2012,7,6,4,-37.945000,-15.989500,12.2,7.6,9.2,132.0,...,175.25,99.91,177.81,97.79,183.18,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
1,2012,7,6,5,-37.945667,-15.988833,12.2,7.4,9.1,122.0,...,189.33,84.58,186.64,97.82,169.48,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
2,2012,7,6,6,-37.945833,-15.987833,12.2,6.2,7.9,127.0,...,325.59,43.52,324.20,43.96,344.74,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
3,2012,7,6,7,-37.945833,-15.988333,12.2,6.8,8.5,112.0,...,270.37,27.47,268.81,28.30,208.79,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
4,2012,7,6,8,-37.945500,-15.989000,12.2,7.6,9.8,116.0,...,212.59,81.56,202.39,78.75,238.80,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0


In [3]:
# H
# nao achei o arquivo po.xlsx, mas parece ser um raw de uma axys historica. O resultado final é um dataframe parseado nos campos de interesse.
# variables vai receber os nomes desses campos de interesse.

## Create DateTime column

In [4]:
df['date_time'] = df['#ano'].astype(str).str.zfill(4) + df['mes'].astype(str).str.zfill(2) + df['dia'].astype(str).str.zfill(2) + df['hora'].astype(str).str.zfill(2)

In [5]:
df['date_time'] = pd.to_datetime(df['date_time'], format='%Y%m%d%H')
df.set_index('date_time', inplace=True)

In [6]:
df.drop(columns=['#ano', 'mes', 'dia', 'hora'], inplace=True)

In [7]:
df.columns = ['lon', 'lat', 'battery', 'wspd1', 'gust1', 'wdir1', 'wspd2', 'gust2',
       'wdir2', 'atmp', 'rh', 'dewpt', 'pres', 'sst', 'compass', 'arad',
       'cspd1', 'cdir1', 'cspd2', 'cdir2', 'cspd3', 'cdir3', 'swvht', 'mxwvht',
       'tp', 'wvdir', 'spred']

In [8]:
df.head()

,lon,lat,battery,wspd1,gust1,wdir1,wspd2,gust2,wdir2,atmp,...,cdir1,cspd2,cdir2,cspd3,cdir3,swvht,mxwvht,tp,wvdir,spred
date_time,,,,,,,,,,,,,,,,,,,,,
2012-07-06 04:00:00,-37.945000,-15.989500,12.2,7.6,9.2,132.0,7.4,9.1,130.0,24.9,...,175.25,99.91,177.81,97.79,183.18,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
2012-07-06 05:00:00,-37.945667,-15.988833,12.2,7.4,9.1,122.0,7.2,8.9,120.0,24.5,...,189.33,84.58,186.64,97.82,169.48,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
2012-07-06 06:00:00,-37.945833,-15.987833,12.2,6.2,7.9,127.0,6.1,7.7,125.0,24.3,...,325.59,43.52,324.20,43.96,344.74,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
2012-07-06 07:00:00,-37.945833,-15.988333,12.2,6.8,8.5,112.0,6.6,8.4,111.0,24.9,...,270.37,27.47,268.81,28.30,208.79,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
2012-07-06 08:00:00,-37.945500,-15.989000,12.2,7.6,9.8,116.0,7.4,9.6,114.0,25.0,...,212.59,81.56,202.39,78.75,238.80,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0


In [9]:
variables = df.drop(columns=['lon', 'lat', 'battery']).columns.to_list()

In [117]:
# H
# obtendo um dado general analogo de uma axys_general_new pra testar e adaptando pra ficar parecido ao po.xlsx

df = pd.read_csv('../itaguai_gen_100.csv',index_col='date_time')

#eventualmente podemos usar o extractor nesse passo

In [11]:
def extractor(id, table='qualified_data.qualified_data'):
    # Connect to your postgres DB
    conn = psycopg2.connect(f"dbname={os.getenv('DB_NAME')} \
                            user={os.getenv('DB_USER')} \
                            password={os.getenv('DB_PASSWORD')} \
                            host={os.getenv('DB_HOST')}")

    query = f"SELECT * FROM {table} WHERE buoy_id = {id}"

    query += " ORDER BY date_time"
    
    df = sqlio.read_sql(query, conn)
    conn.close()

    return df

# df = extractor(8,table='moored.axys_general_new') # exemplo axys
df = extractor(36,table='moored.bmobr_general') # exemplo axys
df.set_index('date_time',inplace=True)

/tmp/ipykernel_15473/3149113173.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = sqlio.read_sql(query, conn)


In [12]:
df.dropna(axis=1,how='all',inplace=True) #retirando as colunas vazias
df.head()

,id,buoy_id,latitude,longitude,geom,battery,temp_datalogger,compass,wspd1,wdir1,...,tsig,h110,t110,tm02,tp_dir,tp_spread,tp5,hm0,te,tm01
date_time,,,,,,,,,,,,,,,,,,,,,
2022-12-06 20:00:00,10426,36,-22.8855,-43.1353,0101000020E61000000C93A982519145C0A69BC420B0E2...,13.8,27.3,NaN,2.8,245.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-06 21:00:00,10427,36,-22.8854,-43.1356,0101000020E610000036AB3E575B9145C0DFE00B93A9E2...,13.1,26.7,NaN,0.2,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-06 22:00:00,10428,36,-22.8854,-43.1356,0101000020E610000036AB3E575B9145C0DFE00B93A9E2...,13.0,26.0,NaN,0.3,218.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-06 23:00:00,10429,36,-22.8854,-43.1356,0101000020E610000036AB3E575B9145C0DFE00B93A9E2...,13.0,25.6,NaN,0.0,185.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-07 00:00:00,10430,36,-22.8854,-43.1356,0101000020E610000036AB3E575B9145C0DFE00B93A9E2...,13.0,25.3,NaN,0.1,32.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df.columns

Index(['id', 'buoy_id', 'latitude', 'longitude', 'geom', 'battery',
       'temp_datalogger', 'compass', 'wspd1', 'wdir1', 'gust1', 'wspd2',
       'wdir2', 'gust2', 'atmp', 'srad', 'rh', 'dewpt', 'pres', 'sst', 'cspd1',
       'cdir1', 'cspd2', 'cdir2', 'cspd3', 'cdir3', 'cspd4', 'cdir4', 'cspd5',
       'cdir5', 'cspd6', 'cdir6', 'cspd7', 'cdir7', 'cspd8', 'cdir8', 'cspd9',
       'cdir9', 'cspd10', 'cdir10', 'cspd11', 'cdir11', 'cspd12', 'cdir12',
       'cspd13', 'cdir13', 'cspd14', 'cdir14', 'cspd15', 'cdir15', 'cspd16',
       'cdir16', 'cspd17', 'cdir17', 'cspd18', 'cdir18', 'swvht1', 'tp1',
       'mxwvht1', 'wvdir1', 'wvspread1', 'swvht2', 'tp2', 'wvdir2', 'avwvht',
       'tav', 'tmax', 'pk_crest', 'tsig', 'h110', 't110', 'tm02', 'tp_dir',
       'tp_spread', 'tp5', 'hm0', 'te', 'tm01'],
      dtype='object')

In [15]:
# adaptando o variable

# para axys
# variables = df.drop(columns=['id','buoy_id','geom','compass','longitude', 'latitude', 'battery']).columns.to_list()

# para bmobr
variables = df.drop(columns=['id','buoy_id','geom','compass','longitude', 'latitude', 'battery','temp_datalogger',
                             'avwvht', 'tav', 'tmax', 'pk_crest', 'tsig', 'h110', 't110', 'tm02', 'tp_dir', 'tp_spread', 'tp5', 'hm0', 'te', 'tm01']).columns.to_list()
variables

['wspd1',
 'wdir1',
 'gust1',
 'wspd2',
 'wdir2',
 'gust2',
 'atmp',
 'srad',
 'rh',
 'dewpt',
 'pres',
 'sst',
 'cspd1',
 'cdir1',
 'cspd2',
 'cdir2',
 'cspd3',
 'cdir3',
 'cspd4',
 'cdir4',
 'cspd5',
 'cdir5',
 'cspd6',
 'cdir6',
 'cspd7',
 'cdir7',
 'cspd8',
 'cdir8',
 'cspd9',
 'cdir9',
 'cspd10',
 'cdir10',
 'cspd11',
 'cdir11',
 'cspd12',
 'cdir12',
 'cspd13',
 'cdir13',
 'cspd14',
 'cdir14',
 'cspd15',
 'cdir15',
 'cspd16',
 'cdir16',
 'cspd17',
 'cdir17',
 'cspd18',
 'cdir18',
 'swvht1',
 'tp1',
 'mxwvht1',
 'wvdir1',
 'wvspread1',
 'swvht2',
 'tp2',
 'wvdir2']

## Open limits for the data

In [57]:
mis_values = pnboia_limits.mis_value_axys_limits
limits = pnboia_limits.range_axys_limits
fine_limits = pnboia_limits.fine_axys_limits
stuck_limit = pnboia_limits.stuck_axys_limits
sigma_values = pnboia_limits.sigma_axys_limits
height = pnboia_limits.height
continuity_limit = pnboia_limits.continuity_axys_limits
anem_limits = pnboia_limits.anemometer_ascat

In [58]:
mis_values

{'rh': [11, -9999, nan],
 'cspd1': [409.5, -9999, nan],
 'cdir1': [511, -9999, nan],
 'cspd2': [409.5, -9999, nan],
 'cdir2': [511, -9999, nan],
 'cspd3': [409.5, -9999, nan],
 'cdir3': [511, -9999, nan],
 'dewpt': [-10, -9999, nan],
 'atmp': [-10, -9999, nan],
 'sst': [40.955, -9999, nan],
 'swvht': [20.47, -9999, nan],
 'mxwvht': [20.47, -9999, nan],
 'tp': [25.5, -9999, nan],
 'wvdir': [381, -9999, nan],
 'spred': [381, -9999, nan]}

In [17]:
sigma_values

{'swvht': 6,
 'rh': 20,
 'pres': 21,
 'atmp': 11,
 'wspd': 25,
 'wspd1': 25,
 'wspd2': 25,
 'sst': 8.6}

In [6]:
# H
# O pnboia_limits tem um qc_config artificial. Em teoria a gente puxa direto do banco ne?
def extractor_qc(id, table='qualified_data.qualified_data'):
    # Connect to your postgres DB
    conn = psycopg2.connect(f"dbname={os.getenv('DB_NAME_LOCAL')} \
                            user={os.getenv('DB_USER')} \
                            password={os.getenv('DB_PASSWORD')} \
                            host=localhost")

    query = f"SELECT * FROM {table} WHERE buoy_id = {id}"

    df = sqlio.read_sql(query, conn)
    conn.close()

    return df

# qc = extractor_qc(9,table='quality_control.general') # axys
qc = extractor_qc(36,table='quality_control.general')
qc_config = qc.qc_config[0]

/tmp/ipykernel_24787/1379919562.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = sqlio.read_sql(query, conn)


In [50]:
# para transformar o qc_config do banco nos equivalentes ao do pnboia_limits
mis_values = {}
limits = {}
fine_limits = {}
stuck_limit = qc_config['stuck_sensor']['limits']
sigma_values = {}
height = qc_config['convert_wind']['height']
continuity_limit = qc_config['t_continuity']['limits']
# anem_limits = ???

In [51]:
for param, limit in zip(qc_config['miss_value']['parameters'], qc_config['miss_value']['limits']):
    mis_values[param] = limit

for param, limit in zip(qc_config['gross_range']['parameters'], qc_config['gross_range']['limits']):
    limits[param] = limit

for param, limit in zip(qc_config['fine_range']['parameters'], qc_config['fine_range']['limits']):
    fine_limits[param] = limit

for param, limit in zip(qc_config['t_continuity']['parameters'], qc_config['t_continuity']['sigma']):
    sigma_values[param] = limit

In [8]:
qc_config.keys()

dict_keys(['miss_value', 'gross_range', 'fine_range', 'stuck_sensor', 'convert_wind', 't_continuity', 'true_north'])

## Create the class to start the qc tests

In [61]:
qc = QCChecks(data=df,
              variables=variables,
              mis_values=mis_values,
              limits=limits,
              fine_limits=fine_limits,
              stuck_limit=stuck_limit,
              sigma_values=sigma_values,
              continuity_limit=continuity_limit,
              height=height
             )

In [60]:
# help(qc)

## Misvalue test

In [62]:
for parameter in mis_values.keys():
    print(parameter)
    qc.mis_value(parameter=parameter)

rh
rh: 1 flagged data
cspd1
cspd1: 0 flagged data
cdir1
cdir1: 0 flagged data
cspd2
cspd2: 0 flagged data
cdir2
cdir2: 0 flagged data
cspd3
cspd3: 0 flagged data
cdir3
cdir3: 0 flagged data
dewpt
dewpt: 0 flagged data
atmp
atmp: 0 flagged data
sst
sst: 0 flagged data
swvht
No mis_value_limit for swvht


KeyError: 'swvht'

In [56]:
mis_values

{'wspd1': [-9999, None],
 'gust1': [-9999, None],
 'wdir1': [-9999, None],
 'wspd2': [-9999, None],
 'gust2': [-9999, None],
 'wdir2': [-9999, None],
 'atmp': [-9999, None],
 'rh': [-9999, None],
 'dewpt': [-9999, None],
 'pres': [-9999, None],
 'sst': [-9999, None],
 'compass': [-9999, None],
 'srad': [-9999, None],
 'cspd1': [-9999, None],
 'cdir1': [-9999, None],
 'cspd2': [-9999, None],
 'cdir2': [-9999, None],
 'cspd3': [-9999, None],
 'cdir3': [-9999, None],
 'cspd4': [-9999, None],
 'cdir4': [-9999, None],
 'cspd5': [-9999, None],
 'cdir5': [-9999, None],
 'cspd6': [-9999, None],
 'cdir6': [-9999, None],
 'cspd7': [-9999, None],
 'cdir7': [-9999, None],
 'cspd8': [-9999, None],
 'cdir8': [-9999, None],
 'cspd9': [-9999, None],
 'cdir9': [-9999, None],
 'cspd10': [-9999, None],
 'cdir10': [-9999, None],
 'cspd11': [-9999, None],
 'cdir11': [-9999, None],
 'cspd12': [-9999, None],
 'cdir12': [-9999, None],
 'cspd13': [-9999, None],
 'cdir13': [-9999, None],
 'cspd14': [-9999, None

In [42]:
# qc.flag.wvdir
# qc.flag.wvdir1

## Range test

In [41]:
# for parameter in limits.keys():
#     qc.range(parameter=parameter, test='gross')
#     qc.range(parameter=parameter, test='fine')

## Comparison between ascat and anemometers


In [40]:
# qc.ascat_anemometer(limits=anem_limits['portoseguro'])

## Comparison between swvht and mxwvht

In [39]:
# qc.swvht_mxwvht(swvht_name = 'swvht1', mxwvht_name = 'mxwvht1')

## Comparison of wind speed and gust

In [38]:
# qc.wind_speed_gust(wspd_name='wspd1', gust_name='gust1')
# qc.wind_speed_gust(wspd_name='wspd2', gust_name='gust2')

## Comparison of Dewpt and Atmp

In [37]:
# qc.dewpt_atmp(dewpt_name='dewpt', atmp_name='atmp')

## Comparison of battery and pressure

In [36]:
# qc.battery(battery_name='battery', pres_name='pres')

## Stuck sensor test

In [35]:
# for parameter in variables:
#     if parameter != 'battery':
#         qc.stuck_sensor(parameter=parameter)

## Convert wind to 10 meters

In [34]:
# qc.convert_wind(wspd_name='wspd1', gust_name='gust1', height=4.7)
# qc.convert_wind(wspd_name='wspd2', gust_name='gust2', height=3.4)

## Select the best anemometer

In [33]:
# qc.best_sensor(parameters1=['wspd1', 'wdir1', 'gust1'],parameters2=['wspd2','wdir2', 'gust2'])

## Time continuity test

In [32]:
# for parameter in sigma_values.keys():
#     try:
#         qc.t_continuity(parameter=parameter)
#     except:
#         print('error')

## Front exception tests

In [31]:
# qc.front_except1(wdir_name='wdir1', atmp_name='atmp')

In [30]:
# qc.front_except3(wspd_name='wspd1', atmp_name='atmp')

In [29]:
# qc.front_except4(pres_name='pres1', wspd_name='wspd1')

In [28]:
# qc.front_except5(pres_name='pres')

In [27]:
# qc.front_except6(wspd_name='wspd', swvht_name='swvht')

## Plots to compare data

In [26]:
# df

In [3]:
# qc.plot_comparison(parameter='wspd1',ylim=[0,15], start_date='2016-01-01', end_date='2024-02-01')

In [2]:
# qc.plot_comparison(parameter='wspd1',
#                    ylim=[0,15],
#                    start_date='2016-01-01',
#                    end_date='2024-02-01',
#                    plot_type='plotly')

In [25]:
# mis_values.values()

In [11]:
anem_test = anem_limits['portoseguro']
qc_config = {
    'miss_value': {
        'parameters': list(mis_values.keys()),
        'limits': list(mis_values.values())
    },
    'gross_range': {
        'parameters': list(limits.keys()),
        'limits': list(limits.values())
    },
    'fine_range':{
        'parameters': list(fine_limits.keys()),
        'limits': list(fine_limits.values())
    },
    'ascat_anemometer':{
        'parameters': [['wspd1', 'wdir1', 'gust1'], ['wspd2','wdir2', 'gust2']],
        'limits': anem_test
    },
    'swvht_mxwvht': {
        'parameters': [['swvht', 'mxwvht']]
    },
    'wind_speed_gust': {
        'parameters': [['wspd1', 'gust1'], ['wspd2', 'gust2']]
    },
    'dewpt_atmp': {
        'parameters': [['dewpt', 'atmp']]
    },
    'battery': {
        'parameters': [['battery', 'pres']]
    },
    'stuck_sensor': {
        'parameters': variables, 'limits': stuck_limit},
    'convert_wind': {
        'parameters': [['wspd1', 'gust1'], ['wspd2', 'gust2']],
        'height': [4.7, 3.4]
    },
    'best_sensor': {
        'parameters': [['wspd1', 'wdir1', 'gust1'], ['wspd2','wdir2', 'gust2']]
    },
    # 't_continuity': {
    #     'parameters': list(sigma_values.keys()),
    #     'sigma': list(sigma_values.values()),
    #     'limits': continuity_limit,
    #     'exceptions': [
    #         {'test': 1,
    #          'parameters': ['wdir', 'atmp']},
    #         {'test': 3,
    #          'parameters': ['wspd', 'atmp']},
    #         {'test': 4,
    #          'parameters': ['pres', 'wspd']},
    #         {'test': 5,
    #          'parameters': 'pres'},
    #         {'test': 6,
    #          'parameters': ['wspd', 'swvht']}
    #     ]
    # },
    'true_north': {
        'parameters': ['wspd1', 'wspd2','cspd1','cdir1','cdir2','cdir3','wvdir'],
        'annual_variation': 0.05,
        'year_reference': 2010,
        'mag_deg': 10.07
    }
}

In [24]:
# qc_config

In [23]:
# qc = QCChecks(data=df,
#               variables=variables,
#               qc_config=qc_config
#              )

In [22]:
# qc.run()